### BIOS
- ***MBR***  $ - $ Master Boot Record   
  - 512 bytes to store boot info
  - 64 bytes to store partitions
  - 3 partitions for file systems, 4th partition is a container for logical
  - Logical partitions (in extended partition *sda4*) - starts with number 5

### UEFI
- ***GPT***  $ - $ GUID Partition Table  
  - 128 partitions.  
  - no logical partitions

___

- `lsblk` $ - $ list block devices
- `cat /proc/partitions` $ - $ all existing block devices in the system
______





## Linux storage options
- Partitions
- LVM Logical Volumes
- Stratis
- Virtual Data Optimizer

## Creating partitions with ***parted***
- `parted /dev/sdb`
- `print`
- `mklabel msdos|gpt`
- `mkpart [part-type name fs-type start end]`
- `mkpart one 1MiB 1024MiB`
- `quit`
- `udevadm settle` $ - $ update devices

## Creating MBR partitions with ***fdisk***
- `fdisk /dev/nvm0n3`
- `partprobe` $ - $ update partition table

## File system differences
- ***XFS***
  - default for RHEL8
  - fast and scalable
  - uses CoW to guarantee data integrity
  - size can be increased, not decreased
- ***Ext4***
  - backward compatible to Ext2
  - uses Journal to guarantee data integrity
  - size can be increased and decreased


## Make and mount file systems
- `mkfs.TAB.TAB` $ - $ see available file systems
- `mkfs.xfs /dev/sdb1`
- `mkfs.ext4`
- `mount /dev/sdb1 /mnt` $ - $ mount temporarily
- `mount | grep '^/'` $ - $ verify
- `umount /dev/sdb1` or `umount /mnt`
- `lsof /mnt` $ - $ list open files in /mnt
- 

## Mount partitions using ***/etc/fstab***
- `/etc/fstab` $ - $ main config file to persistently mount partitions. Used to generate systemd mounts by the **systemd-fstab-generator** utility
- - `/dev/sdb1   /xfs   xfs   defaults    0 0` $ - $ add this line
- `systemctl daemon-reload` $ - $ update systemd after editing /etc/fstab
- `mkdir /xfs` $ - $ create mountpoints
- `mount -a ` $ - $ mount all file systems

## Managing persistent naming attributes
###***Problem***
When we make logical partitions, we see many device names such as `/dev/nvm0n3p6`. But they may change. The last symbol stands for the number of logical partition and it is generated automatically every time we do `mount -a`, the first will be always 5. So there can be a conflict: there first may be 5, but we assigned 6. To avoid it during modifing /`etc/fstab`, use *LABEL* (unique device name) instead of `/dev/nvm0n3p6`, where the last digit is always different depending on how many partitions we have to mount before current.
###***Demonstration***
1. Create new partitions:
  - `fdisk /dev/sdb`
2. Create file systems for them:
  - `mkfs.xfs /dev/sdb5`
  - `mkfs.ext4 /dev/sdb6`
3. Create directories for file systems:
  - `mkdir /books /articles`
4. Edit `/etc/fstab`
  - `/dev/sdb5 /books xfs defaults 0 0`
  - `/dev/sdb6 /articles ext4 defaults 0 0`
5. `mount -a`
6. Delete the 5th logical partition:
  - `fdisk /dev/sdb`
  - `d`
  - `5`
  - `w`
7. Error "device busy"
  - comment line with 5th partition in `/etc/fstab`
8. Reboot.
  - Press 'e' to edit. Erase "rhgb quiet" to see how the Linux boots.
  - During booting there will be en error saying it cann't access /dev/sdb6.
  - Emergency mode is loaded. Enter root password.
  - Comment line with 6th partition in `/etc/fstab`
  - Ctrl+D to normal operational mode.
9. `lsblk`
10. `blkid` $ - $ block id, lists UUID and LABEL
11. ***tune2fs*** and ***xfs_admin*** utilities
  - `tune2fs -L articles /dev/sdb5`
  - `LABEL=articles /articles ext4 default 0 0` $ - $ in `/etc/fstab`
  - `mount -a`
  - `mount | grep artic`


___________________

- ***UUID*** $ - $ autogenerated for each device with file system
- ***LABEL*** $ - $ while creating file system, use option `-L`
___________________
### System unique device names
- `cd /dev/disk`
- `ls by-path`


## Managing systemd mounts
- /etc/fstab mounts are already systemd mounts
- mounts can be created using systemd .mount files
- using .mount files allows you to be more specific in defining dependencies
- `systemctl cat tmp.mount` $ - $ example
- `mount -a | grep tmp` $ - $ won't work
- `systemctl enable --now tmp.mount`
- `mount | grep tmp`  

Make persistent:  
- Comment line for articles from /etc/fstab
- `cp /usr/lib/systemd/system/tmp/mount /etc/systemd/system/articles.mount` (name should reflect the name of the mount directory. For /data/articles should be data-articles.mount)
- 
```
[Mount]
What=LABEL=articles
Where=/articles
Type=ext4
Options=defaults
```
- `systemctl daemon-reload` $ - $ tell the system about new .mount file
- `systemctl status articles.mount` $ - $ loaded(/proc/self/mountinfo), this file was autogenerated by fstab, because fs has already been mounted with /etc/fstab
- `umount /articles`
- `systemctl enable --now articles.mount`
- `systemctl status articles.mount` $ - $ correct

## Managing XFS
####***xfsdump***
- creates backups
- `xfsdump -l 0 -f /backupfiles/data.xfsdump /data`

####***xfsrestore***
- restores backups
- `xfsrestore -f /backupfiles/data.xfsdump /data`

####***xfsrepair***


## Swap
Swap is RAM emulated on disk. Can be created on any block device, including swap files.  
####***parted:***
- `parted /dev/sdb`
- `mkpart *enter*`
  - name: swap
  - file system: linux-swap 
  - start
  - end
- `lsblk` $ - $ check our new partition name (f.e. /dev/sdb2)
- `mkswap /dev/sdb2` $ - $ create swap file system
- `free -m` $ - $ see current swap space
- `swapon` $ - $ activate
- `free -m` $ - $ see swap space is bigger  

But its still not persistant. To make mount persistant do:
- `/dev/sdb2 swap swap defaults 0 0` in /etc/fstab
- `systemctl daemon-reload`

####***fdisk:***
- partition type *82*